In [22]:
%matplotlib inline
import numpy as np
import random
import json
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
stop = set(stopwords.words('english'))

In [23]:
reviews = list(open('yelp_dataset/yelp_reviews.json'))
json_reviews = [json.loads(rev) for rev in reviews[:50000]]

In [24]:
train_set = json_reviews[:int(len(json_reviews)*0.25)]
train_set = [(rev['text'].lower().replace('\n', '').split(' '), rev['stars']) for rev in train_set if rev['stars'] != 3]

In [25]:
train_pos_text = []
train_neg_text = []

train_pos_avg_word_len = []
train_pos_avg_rev_len = []

train_neg_avg_word_len = []
train_neg_avg_rev_len = []

for review in train_set:
    if review[1] > 3:
        train_pos_text.append(review)
    elif review[1] < 3:
        train_neg_text.append(review)

In [26]:
# threshold for positive stop word review length vs negative stop word review length (below positive, above negative)
pos_stops = []
for review in train_pos_text:
    pos_stops.append((len([word for word in review[0] if word in stop]), review[1]))
    
neg_stops = []
for review in train_neg_text:
    neg_stops.append((len([word for word in review[0] if word in stop]), review[1]))

In [27]:
# list of positive words
poswords = list(open('poswords.txt', 'r'))
poswords = [word.split('\n')[0] for word in poswords]

#list of negative words
negwords = list(open('negwords.txt', 'r'))
negwords = [word.split('\n')[0] for word in negwords]

In [28]:
scores = []
for each in train_set:
    avg_word_len = np.mean([len(word) for word in each[0]])
    rev_len = len(each[0])
    stop_count = len([word for word in each[0] if word in stop])
    pos_word_count = len([word for word in each[0] if word in poswords])
    neg_word_count = len([word for word in each[0] if word in negwords])
    scores.append([avg_word_len, rev_len, stop_count, pos_word_count, neg_word_count])

In [29]:
test_set = json_reviews[int(len(json_reviews)*0.25):]
test_set = [(rev['text'].lower().replace('\n', '').split(' '), rev['stars']) for rev in test_set if rev['stars'] != 3]

In [30]:
guess_scores = []
for each in test_set:
    avg_word_len = np.mean([len(word) for word in each[0]])
    rev_len = len(each[0])
    stop_count = len([word for word in each[0] if word in stop])
    pos_word_count = len([word for word in each[0] if word in poswords])
    neg_word_count = len([word for word in each[0] if word in negwords])
    guess_scores.append([avg_word_len, rev_len, stop_count, pos_word_count, neg_word_count])

In [31]:
log_train = []
for each in train_set:
    if each[1] > 3:
        log_train.append((each[0], 1))
    elif each[1] < 3:
        log_train.append((each[0], 0))
        
log_test = []
for each in test_set:
    if each[1] > 3:
        log_test.append((each[0], 1))
    elif each[1] < 3:
        log_test.append((each[0], 0))

In [32]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(fit_intercept=False, C = 1e9)
mdl = model.fit(scores, np.array([e[1] for e in log_train]))

In [33]:
correct = 0
for actual, guess in zip(log_test, guess_scores):
    predict = float(mdl.predict(guess))
    if actual[1] == predict:
        correct +=1

In [34]:
correct/len(test_set) * 100

81.45344436033308